In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
from PIL import Image
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model('../input/catordog224/catordogresNet.h5')

In [ ]:
from keras.models import load_model
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
img_predict = []
for image_name in tqdm(os.listdir('../input/petfinder-pawpularity-score/train')):
    try:
        img = Image.open(os.path.join('../input/petfinder-pawpularity-score/train', image_name))  # 画像読み込
        img = img.convert("RGB")          
        img = img.resize((224, 224))  # 画像のリサイズ
        img_array = np.asarray(img)
        img_predict.append(img_array)        
    except Exception as e:
        pass

img_predict = np.asarray(img_predict)

In [ ]:
class TQDMPredictCallback(keras.callbacks.Callback):
    def __init__(self, custom_tqdm_instance=None, tqdm_cls=tqdm, **tqdm_params):
        super().__init__()
        self.tqdm_cls = tqdm_cls
        self.tqdm_progress = None
        self.prev_predict_batch = None
        self.custom_tqdm_instance = custom_tqdm_instance
        self.tqdm_params = tqdm_params

    def on_predict_batch_begin(self, batch, logs=None):
        pass

    def on_predict_batch_end(self, batch, logs=None):
        self.tqdm_progress.update(batch - self.prev_predict_batch)
        self.prev_predict_batch = batch

    def on_predict_begin(self, logs=None):
        self.prev_predict_batch = 0
        if self.custom_tqdm_instance:
            self.tqdm_progress = self.custom_tqdm_instance
            return

        total = self.params.get('steps')
        if total:
            total -= 1

        self.tqdm_progress = self.tqdm_cls(total=total, **self.tqdm_params)

    def on_predict_end(self, logs=None):
        if self.tqdm_progress and not self.custom_tqdm_instance:
            self.tqdm_progress.close()

In [ ]:
result_predict = model.predict(img_predict, callbacks=[TQDMPredictCallback()])

In [ ]:
result_predict

In [ ]:
import tqdm
import pandas as pd

df_animal = pd.DataFrame(result_predict)

In [ ]:
df_animal

In [ ]:
df_animal.columns = ['dog','cat']

In [ ]:
df_animal

In [ ]:
df_animal.loc[df_animal['dog'] > 0.5, 'dog'] = 1

In [ ]:
df_animal.loc[df_animal['cat'] > 0.5, 'cat'] = 1

In [ ]:
df_int = df_animal.astype('int')
df_int

In [ ]:
import csv

In [ ]:
df_pet=pd.read_csv("../input/petfinder-pawpularity-score/train.csv")

In [ ]:
df_pet

In [ ]:
df_concat = pd.concat([df_pet , df_int], axis=1)

In [ ]:
df_concat

In [ ]:
# Regression Models
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, VotingRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Evalution Metrix
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
def rmse_score(y_label, y_preds):
    """
    Gives RMSE score 評価指標の一つ　平均２乗誤差の平方根
    """
    return np.sqrt(mean_squared_error(y_label, y_preds))
    

def trainRegModels(df : "data_file", features : list, label: str):
    """
    To automate the training of regression models. Considering
        > RMSE
        > R2 score
    
    """
    regModels = {
            "LinearRegression": LinearRegression(),
            "KNeighborsRegressor": KNeighborsRegressor(n_neighbors=2),
            "AdaBoostRegressor": AdaBoostRegressor(random_state=0, n_estimators=100),
            "LGBMRegressor": LGBMRegressor(),
            "Ridge": Ridge(alpha=1.0),
            "ElasticNet": ElasticNet(random_state=0),
            "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
            "DecisionTreeRegressor": DecisionTreeRegressor(),
            "ExtraTreesRegressor": ExtraTreesRegressor(n_jobs=-1),
            "RandomForestRegressor": RandomForestRegressor(n_jobs=-1),
            "XGBRegressor": XGBRegressor(n_jobs=-1),
            "CatBoostRegressor": CatBoostRegressor(iterations=900, depth=5, learning_rate=0.05, loss_function = 'RMSE'),
        }
    
    # Will return this as a data frame
    summary = {
        "Model" : [],
        "Avg R2 Train Score" : [],
        "Avg R2 Val Score" : [],
        "Avg RSME Train Score" : [],
        "Avg RSME Val Score" : []
    }
    
    # Training
    for idx in trange(len(regModels.keys()), desc = "Models are training...", bar_format="{l_bar}%s{bar:50}%s{r_bar}" % (Fore.CYAN, Fore.RESET), position = 0, leave = True):
        name = list(regModels.keys())[idx]
        model = regModels[name]
        
        # Initializing all the scores to 0
        r2_train = 0; r2_val = 0
        rmse_train = 0; rmse_val = 0
        
        # Running K-fold Cross-validation on every model
        for fold in range(5):
            train_df = df.loc[df.kfold != fold].reset_index(drop = True)
            val_df = df.loc[df.kfold == fold].reset_index(drop = True)
            #print(train_df)
            #print(val_df)
            
            
            train_X = train_df[features]; train_Y = train_df[label]
            val_X = val_df[features]; val_Y = val_df[label]
            #print(val_X)
            
            cur_model = model
            if name == 'CatBoostRegressor':
                cur_model.fit(train_X, train_Y,verbose=False)
            else:
                cur_model.fit(train_X, train_Y)

            Y_train_preds = model.predict(train_X)
            Y_val_preds = model.predict(val_X)
            
            # Collecting the scores
            r2_train += r2_score(train_Y, Y_train_preds)
            r2_val += r2_score(val_Y, Y_val_preds)
            
            rmse_train += rmse_score(train_Y, Y_train_preds)
            rmse_val += rmse_score(val_Y, Y_val_preds)
        
        # Pushing the scores and the Model names
        summary["Model"].append(name)
        #kfoldが５回だから５で割る
        summary["Avg R2 Train Score"].append(r2_train/5)
        summary["Avg R2 Val Score"].append(r2_val/5)
        summary["Avg RSME Train Score"].append(rmse_train/5)
        summary["Avg RSME Val Score"].append(rmse_val/5)
    
    # Finally returning the summary dictionary as a dataframe
    summary_df = pd.DataFrame(summary)
    return summary_df

In [ ]:
req_cols = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur','dog','cat'
]

In [ ]:
def create_folds_regression(data, target="target", num_splits = 5): 
    """
    Helper function to create folds
    
    """
    data["kfold"] = -1 
    data = data.sample(frac=1).reset_index(drop=True)
    #fracは０～１で元データの割合を示している　１なのでランダムにすべての要素をもってくる
    
    # Applying Sturg's rule to calculate the no. of bins for target
    #スタージェスの法則　ヒストグラムを作成するときに階級の数を決定する目安を得られる公式
    num_bins = int(1 + np.log2(len(data))) 

    #pd.cut　境界値によって分割　
    data.loc[:, "bins"] = pd.cut(data[target], bins=num_bins, labels=False) 
    
    #Kfoldの種類
    kf = StratifiedKFold(n_splits=num_splits)
    
    #enumerate　インデックス番号と要素を得ることができる
    #kf.splitはStratifiedKFoldに沿って分割を行っているだけ
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)): 
        data.loc[v_, 'kfold'] = f
        
    data = data.drop(["bins"], axis = 1)         
    return data 

In [ ]:
data_df = create_folds_regression(df_concat, target = 'Pawpularity', num_splits = 5)
data_df.kfold.value_counts()

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from colorama import Fore
from glob import glob
import json
from pprint import pprint
import time
import cv2
from enum import Enum
from IPython.display import display

# For Data preparation
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *


import warnings
warnings.filterwarnings("ignore")

training_summary = trainRegModels(data_df, req_cols, "Pawpularity")
training_summary

In [ ]:
training_summary.sort_values("Avg RSME Val Score", axis = 0, ascending = True)

In [ ]:
#testデータ
df_test=pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

In [ ]:
df_test

In [ ]:
from keras.models import load_model
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
test_img_predict = []
for image_name in tqdm(os.listdir('../input/petfinder-pawpularity-score/test')):
    try:
        img = Image.open(os.path.join('../input/petfinder-pawpularity-score/test', image_name))  # 画像読み込
        img = img.convert("RGB")          
        img = img.resize((224, 224))  # 画像のリサイズ
        img_array = np.asarray(img)
        test_img_predict.append(img_array)        
    except Exception as e:
        pass

test_img_predict = np.asarray(test_img_predict)

In [ ]:
test_img_predict

In [ ]:
test_result_predict = model.predict(test_img_predict, callbacks=[TQDMPredictCallback()])

In [ ]:
df_test_animal = pd.DataFrame(test_result_predict)

In [ ]:
df_test_animal

In [ ]:
df_test_animal.columns = ['dog','cat']


In [ ]:
df_test_animal

In [ ]:
df_test_animal.loc[df_test_animal['dog'] > 0.5, 'dog'] = 1
df_test_animal.loc[df_test_animal['cat'] > 0.5, 'cat'] = 1
df_test_int = df_test_animal.astype('int')
df_test_int

In [ ]:
dfdf_test = pd.concat([df_test , df_test_int], axis=1)

In [ ]:
dfdf_test

In [ ]:
en = ElasticNet(random_state=0)
gbr = GradientBoostingRegressor(random_state=0)
VR_model = VotingRegressor([('en', en),('gbr', gbr)], n_jobs=-1)

r2_train = 0; r2_val = 0
rmse_train = 0; rmse_val = 0

model = VR_model
for fold in trange(5, desc = "Models are training...", bar_format="{l_bar}%s{bar:50}%s{r_bar}" % (Fore.CYAN, Fore.RESET), position = 0, leave = True):
    train_df = data_df.loc[data_df.kfold != fold].reset_index(drop = True)
    val_df = data_df.loc[data_df.kfold == fold].reset_index(drop = True)

    train_X = train_df[req_cols]; train_Y = train_df["Pawpularity"]
    val_X = val_df[req_cols]; val_Y = val_df["Pawpularity"]
    
    model.fit(train_X, train_Y)

    Y_train_preds = model.predict(train_X)
    Y_val_preds = model.predict(val_X)

    # Collecting the scores
    r2_train += r2_score(train_Y, Y_train_preds)
    r2_val += r2_score(val_Y, Y_val_preds)

    rmse_train += rmse_score(train_Y, Y_train_preds)
    rmse_val += rmse_score(val_Y, Y_val_preds)

print(f"Avg R2 Train Score : {r2_train/5}")
print(f"Avg R2 Val Score : {r2_val/5}")
print(f"Avg RSME Train Score : {rmse_train/5}")
print(f"Avg RSME Val Score : {rmse_val/5}")

In [ ]:
test_X = dfdf_test[req_cols]

model_preds = model.predict(test_X)
dfdf_test["Pawpularity"] = model_preds

submission = dfdf_test[["Id", "Pawpularity"]]
submission.to_csv("submission.csv", index = False)
data_df.to_csv("data.csv", index = False)
dfdf_test.to_csv("test.csv", index = False)
submission